# Parameters

The computational and physical parameters are handled at `class xcompact3d_toolbox.Parameters`. It is built on top of [Traitlets](https://traitlets.readthedocs.io/en/stable/index.html), which brings some advantages:

* Attributes are type-checked;
* Default values and restrictions (min and max) are applied where necessary;
* 'On change' callbacks for validation and observation;
* Two-way linking with [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/).

In [1]:
import xcompact3d_toolbox as x3d
import numpy as np

Use `np.float64` if Xcompact3d was compiled with the flag `-DDOUBLE_PREC`, use `np.float32` otherwise.

In [2]:
x3d.mytype = np.float32

## Initialization

There are a few ways to initialize the class. First, calling it with no arguments initializes all variables with default value:

In [3]:
prm = x3d.Parameters()

You can access a list with all the available variables at [Api reference](file:///C:/Users/felip/Documents/GitHub/Xcompact3d-toolbox/docs/_build/html/Docstrings.html#xcompact3d_toolbox.parameters.Parameters).

It is possible to access and/or set values afterwards:

In [4]:
# Reynolds Number
print(prm.re)
prm.re = 1e6
print(prm.re)

1000.0
1000000.0


Second, we can specify some values, and let the missing ones be initialized with default value:

In [5]:
prm = x3d.Parameters(
    filename = 'example.i3d',
    itype = 10,
    nx = 257,
    ny = 129,
    nz = 32,
    xlx = 15.0,
    yly = 10.0,
    zlz = 3.0,
    nclx1 = 2,
    nclxn = 2,
    ncly1 = 1,
    nclyn = 1,
    nclz1 = 0,
    nclzn = 0,
    iin = 1,
    re = 300.0,
    init_noise = 0.0125,
    inflow_noise = 0.0125,
    dt = 0.0025,
    ifirst = 1,
    ilast = 45000,
    irestart = 0,
    icheckpoint = 45000,
    ioutput = 200,
    iprocessing = 50,
)

It is easy to write `example.i3d` to disc, just type:

In [6]:
prm.write()

And finally, it is possible to read the parameters from the disc:

In [7]:
prm = x3d.Parameters(filename = 'example.i3d')
prm.read()

## Traitlets

### Type-checking

All parameters are type-checked, to make sure that they are what `Xcompact3d` expects. Use the cellcode below to see how a `TraitError` pops out when we try:

```python
prm.itype = 10.5
prm.itype = -5
prm.itype = 12
prm.itype = 'sandbox'

```


### Validation

Some parameters, like mesh points (`nx`, `ny` and `nz`), trigger a validation operation when a new value is attributed to them.
Due to restrictions at the FFT library, they must be equal to:

$$
n_i = \left\{ \begin{array}{ll} 2^{1+a} \times 3^b \times 5^c &\mbox{if periodic,} \\ 2^{1+a} \times 3^b \times 5^c + 1 &\mbox{otherwise,}
\end{array} \right.
$$

where $a$, $b$ and $c$ are non negative integers. In addition, the derivatives stencil imposes that:

$$
n_i \ge \left\{ \begin{array}{ll} 8 &\mbox{if periodic,} \\ 9 &\mbox{otherwise.}
\end{array} \right.
$$

Again, give it a try at the cellcode below:

```python
prm.nx = 129
prm.nx = 4
prm.nx = 60
prm.nx = 61
```

### Observation

Other parameters, like mesh resolution (`dx`, `dy` and `dz`), are automaticaly updated when any new atribution occurs to mesh points and/or domain size. Let's create a quick print functions to play with:

In [8]:
def show_param():
    for var in 'nclx1 nclxn nx xlx dx'.split():
        print(f"{var:>5} = {getattr(prm, var)}")

We are starting with:

In [9]:
show_param()

nclx1 = 2
nclxn = 2
   nx = 257
  xlx = 15.0
   dx = 0.05859375


Let's change just the domain's length:

In [10]:
prm.xlx = 50.0

show_param()

nclx1 = 2
nclxn = 2
   nx = 257
  xlx = 50.0
   dx = 0.1953125


The resolutions was updated as well. Now the number of mesh points:

In [11]:
prm.nx = 121

show_param()

nclx1 = 2
nclxn = 2
   nx = 121
  xlx = 50.0
   dx = 0.4166666666666667


Again, the resolutions was updated. Now we set a new mesh resolution, this time, `xlx` will be updated in order to satisfy the new resolution:

In [12]:
prm.dx = 1e-2

show_param()

nclx1 = 2
nclxn = 2
   nx = 121
  xlx = 1.2
   dx = 0.01


Boundary conditions are observed as well. Xcompact3d allows three different BC for velocity:

* Periodic `0`;
* Free-slip `1`;
* Dirichlet `2`.

They can be assigned individualy for each of the six boundaries:

* `nclx1` and `nclxn`, where $x=0$ and $x=xlx$;
* `ncly1` and `nclyn`, where $y=0$ and $y=yly$;
* `nclz1` and `nclzn`, where $z=0$ and $z=zlz$.

It leads to 5 possibilities (`00`, `11`, `12`, `21` and `22`), because both boundary must be periodic, or not, so `0` cannot be combined.

Let's check it out, we are starting with:

In [13]:
show_param()

nclx1 = 2
nclxn = 2
   nx = 121
  xlx = 1.2
   dx = 0.01


We will change just one side to periodic (`nclx1 = 0`), for consistence, the other side should be periodic too. Let's see:

In [14]:
prm.nclx1 = 0

show_param()

nclx1 = 0
nclxn = 0
   nx = 120
  xlx = 1.2
   dx = 0.01


Now free-slip in one side (`nclx1 = 1`), and the other should be non-periodic:

In [15]:
prm.nclx1 = 1

show_param()

nclx1 = 1
nclxn = 1
   nx = 121
  xlx = 1.2
   dx = 0.01


Setting the other boundary to periodic:

In [16]:
prm.nclxn = 0

show_param()

nclx1 = 0
nclxn = 0
   nx = 120
  xlx = 1.2
   dx = 0.01


and now back to Dirichlet:

In [17]:
prm.nclxn = 2

show_param()

nclx1 = 2
nclxn = 2
   nx = 121
  xlx = 1.2
   dx = 0.01


This time, free-slip:

In [18]:
prm.nclxn = 1

show_param()

nclx1 = 2
nclxn = 1
   nx = 121
  xlx = 1.2
   dx = 0.01


There was no need to update `nclx1`, because `1` and `2` can be freely combined. Notice that `nx` was modified properly from 121 to 120 and then back, according to the possible values, `dx` and `xlx` stayed untouched.

### Metadata

Traitlets types constructors have a `tag` method to store metadata in a dictionary. In the case of Xcompact3d-toolbox, two are especially useful:

* `group` defines to what namespace a given parameter belongs when the class is written to `.i3d` file (`.write()` method) or read from `.i3d` file (`.read()` method), parameters without a group are ignored for both methods;
* `widget` defines an [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) used to interact with each parameter.

### Linked widget

<div class="alert alert-warning">

There is a bug reported affecting this subject [#2](https://github.com/fschuch/xcompact3d_toolbox/issues/2), the widgets are not working properly at the moment.

</div>

While the problem above persists, we can activate the link manually for this tutorial:

In [19]:
prm.link_widgets()

Parameters with `widget` tag are related to the value at its widget in a two-way link. Widgets are returned on demand when `class Parameters` is called, let's see:

In [20]:
prm('nx', 'xlx', 'dx', 'nclx1', 'nclxn')

You can play around with the widgets above and see the effect of the observations made previously.

Since there is a two-way link, a `print` will show the actual value on the widgets:

In [21]:
show_param()

nclx1 = 2
nclxn = 1
   nx = 121
  xlx = 1.2
   dx = 0.01


Give it a try, modify the values at the widgets and print them again.

It also works on the other way, atribute a new value to a parameters will change its widget, try it:

In [22]:
#prm.nclx1 = 0

And of course, different widgets for the same parameter are always synchronized, change the widget bellow and see waht hapens with the widget above:

In [23]:
prm('nx')

## Graphical User Interface

To conclude this part of the tutorial, let's see what happend when `class Parameters` is called with no arguments, hover the mouse over some variable to see its description:

In [24]:
prm()